In [1]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
dataset = "Liver"
datatype = "log2"
datafolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\data\\"+dataset+"\\"
resfolder =  "C:\\Users\\amruk\\source\\enrichment-auc\\results\\"+dataset+"\\"+datatype+"\\"

## Geneset - remove empty IDs

In [3]:
geneset_info = pd.read_csv(datafolder+"genesets_modules.csv", index_col=0)
geneset_info.tail()

,ID,Title,source,DataBase
210,Endothelial cell,Endothelial cell,CM,CellMarker
35,Kupffer cell,Kupffer cell,CM,CellMarker
41,Liver bud hepatic cell,Liver bud hepatic cell,CM,CellMarker
51,Mesenchymal cell,Mesenchymal cell,CM,CellMarker
61,T cell,T cell,CM,CellMarker


In [4]:
geneset_info['ID'] = np.where(geneset_info['ID'] == "-", geneset_info['Title'], geneset_info['ID'])
geneset_info.tail()

,ID,Title,source,DataBase
210,Endothelial cell,Endothelial cell,CM,CellMarker
35,Kupffer cell,Kupffer cell,CM,CellMarker
41,Liver bud hepatic cell,Liver bud hepatic cell,CM,CellMarker
51,Mesenchymal cell,Mesenchymal cell,CM,CellMarker
61,T cell,T cell,CM,CellMarker


In [5]:
geneset_info.to_csv(datafolder+ "genesets_modules.csv")

## Filter geneset by occuring genes

In [6]:
geneset_info = pd.read_csv(datafolder+"genesets_modules.csv", index_col=0)
gene_expr = pd.read_csv(datafolder + datatype + "_filtered_data.csv", index_col=0)
patients_names = gene_expr.columns.to_list()
genes = gene_expr.index.tolist()
gene_expr = gene_expr.to_numpy().astype(float)

In [7]:
with open(datafolder + "genesets_genes.json") as file:
    genesets = json.load(file)
gs_names = list(genesets.keys())

In [8]:
incorrect = []
for i, (gs_name, geneset_genes) in tqdm(enumerate(genesets.items()), total=len(genesets)):
    genes_in_ds = [gene in geneset_genes for gene in genes]
    in_gs = gene_expr[genes_in_ds, :]
    N_gs = in_gs.shape[0]  # number of genes in GS
    N_tot = len(genes)  # total number of genes
    if N_gs == 0:
        incorrect.append(gs_name)

100%|██████████| 168/168 [00:03<00:00, 44.05it/s] 


In [9]:
filtered_geneset_info = geneset_info[~geneset_info['ID'].isin(incorrect)]
filtered_genesets = {key: genesets[key] for key in genesets.keys() if key not in incorrect}

In [11]:
len(filtered_genesets)

160

In [12]:
filtered_geneset_info.groupby(["DataBase"]).size()

DataBase
CIBERSORT     45
CellMarker     6
KEGG          29
MSigDB        34
PanglaoDB     13
tmod          26
dtype: int64

In [13]:
filtered_geneset_info.to_csv(datafolder+ "genesets_modules.csv")
with open(datafolder + "genesets_genes.json", 'w') as fp:
    json.dump(filtered_genesets, fp)